In [270]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import random
from scipy.optimize import minimize

In [271]:
def generate_data():

  """ Generate data to be fitted by multiple regression """

  # Set random seed
  random.seed(0)

  # Generate 4 dimensional data (3 inputs + 1 output)
  # Inputs are all random number in range [0, 1]
  # Output depends linearly on some input parameters
  npoint = 50
  x1 = random.rand(npoint)
  x2 = random.rand(npoint)
  x3 = random.rand(npoint)
  noise = random.randn(npoint) * 0.1
  y = 1.0 + 2.0*x1 + 3.0*x2 + noise

  return x1, x2, x3, y

In [272]:
def rss_3param(p, x1, x2, x3, y):
  """ Calculate RSS for 3-input fit, to be minimized """
  ymodel = p[0] + p[1]*x1 + p[2]*x2 + p[3]*x3
  rss = np.sum((y - ymodel)**2)
  return rss

In [273]:
def fit_3param(x1, x2, x3, y):
  """ Fit 3-input function to data, return RSS and best-fit parameters """
  p0 = [1.0, 1.0, 1.0, 1.0]
  out = minimize(rss_3param, p0, args=(x1, x2, x3, y))
  rss = rss_3param(out.x, x1, x2, x3, y)
  return rss, out.x

In [274]:
def rss_2param(p, x1, x2, y):
  """ Calculate RSS for 2-input fit, to be minimized """
  ymodel = p[0] + p[1]*x1 + p[2]*x2
  rss = np.sum((y - ymodel)**2)
  return rss

In [275]:
def fit_2param(x1, x2, y):
  """ Fit 2-input function to data, return RSS and best-fit parameters """
  p0 = [1.0, 1.0, 1.0]
  out = minimize(rss_2param, p0, args=(x1, x2, y))
  rss = rss_2param(out.x, x1, x2, y)
  return rss, out.x

In [276]:
def rss_1param(p, x, y):
  """ Calculate RSS for 1-input fit, to be minimized """
  ymodel = p[0] + p[1]*x
  rss = np.sum((y - ymodel)**2)
  return rss

In [277]:
def fit_1param(x, y):
  """ Fit 1-input function to data, return RSS and best-fit parameters """
  p0 = [1.0, 1.0]
  out = minimize(rss_1param, p0, args=(x, y))
  rss = rss_1param(out.x, x, y)
  return rss, out.x

In [278]:
def rss_0param(p, y):
  """ Calculate RSS for 0-input (constant) fit, to be minimized """
  ymodel = p[0]
  rss = np.sum((y - ymodel)**2)
  return rss

In [279]:
def fit_0param(y):
  """ Fit 0-input (constant) to data, return RSS and best-fit parameters """
  p0 = [1.0]
  out = minimize(rss_0param, p0, args=(y))
  rss = rss_0param(out.x, y)
  return rss, out.x

In [280]:
def cp(rss, n, d, sigma_square):
  """ Calculate the Mallows's Cp parameter """
  cp = (rss + 2.0 * d * sigma_square) / n
  return cp

In [281]:
def aic(rss, n, d, sigma_square):
  """ Calculate the Akaike Information Criteria (AIC) """
  aic = (rss + 2.0 * d * sigma_square) / n / sigma_square
  return aic

In [282]:
def bic(rss, n, d, sigma_square):
  """ Calculate the Bayesian Information Criteria (BIC) """
  bic = (rss + np.log(n) * d * sigma_square) / n / sigma_square
  return bic

In [283]:
def subset_selection():

  """ Main function to illustrate subset selection.

  This function takes 4-dimensional data (3 inputs, 1 output) and fit the
  data with all possible combinations of inputs (total of 8 combinations).
  For each fit, the best-fit parameters, RSS, AIC and BIC are printed out.
  """

  # Get the data
  x1, x2, x3, y = generate_data()

  # Header of the table format and formatted string to print output
  # Read "Format Specification Mini-Language" for more details
  headstr = "Model      Beta0   Beta1   Beta2   Beta3     RSS      AIC      BIC"
  print(headstr)
  print("="*len(headstr))
  outstr = "{0:8s}  {1:6.3f}  {2:6.3f}  {3:6.3f}  {4:6.3f}  {5:6.3f}  {6:7.3f}  {7:7.3f}"

  # Model with all 3 inputs
  rss123, p123 = fit_3param(x1, x2, x3, y)
  sigma_square = rss123 / len(y)
  cp123 = cp(rss123, len(y), 4, sigma_square)
  aic123 = aic(rss123, len(y), 4, sigma_square)
  bic123 = bic(rss123, len(y), 4, sigma_square)
  print(outstr.format("x1/x2/x3", p123[0], p123[1], p123[2], p123[3], rss123, aic123, bic123))

  # Models with 2 inputs (3 variations)
  rss12, p12 = fit_2param(x1, x2, y)
  aic12 = aic(rss12, len(y), 3, sigma_square)
  bic12 = bic(rss12, len(y), 3, sigma_square)
  rss13, p13 = fit_2param(x1, x3, y)
  aic13 = aic(rss13, len(y), 3, sigma_square)
  bic13 = bic(rss13, len(y), 3, sigma_square)
  rss23, p23 = fit_2param(x2, x3, y)
  aic23 = aic(rss23, len(y), 3, sigma_square)
  bic23 = bic(rss23, len(y), 3, sigma_square)
  print(outstr.format("x1/x2", p12[0], p12[1], p12[2], 0.0, rss12, aic12, bic12))
  print(outstr.format("x1/x3", p13[0], p13[1], 0.0, p13[2], rss13, aic13, bic13))
  print(outstr.format("x2/x3", p23[0], 0.0, p23[1], p23[2], rss23, aic23, bic23))

  # Models with 1 inputs (3 variations)
  rss1, p1 = fit_1param(x1, y)
  aic1 = aic(rss1, len(y), 2, sigma_square)
  bic1 = bic(rss1, len(y), 2, sigma_square)
  rss2, p2 = fit_1param(x2, y)
  aic2 = aic(rss2, len(y), 2, sigma_square)
  bic2 = bic(rss2, len(y), 2, sigma_square)
  rss3, p3 = fit_1param(x3, y)
  aic3 = aic(rss3, len(y), 2, sigma_square)
  bic3 = bic(rss3, len(y), 2, sigma_square)
  print(outstr.format("x1", p1[0], p1[1], 0.0, 0.0, rss1, aic1, bic1))
  print(outstr.format("x2", p2[0], 0.0, p2[1], 0.0, rss2, aic2, bic2))
  print(outstr.format("x3", p3[0], 0.0, 0.0, p3[1], rss3, aic3, bic3))

  # Model with zero inputs
  rss0, p0 = fit_0param(y)
  aic0 = aic(rss0, len(y), 1, sigma_square)
  bic0 = bic(rss0, len(y), 1, sigma_square)
  print(outstr.format("Const", p0[0], 0.0, 0.0, 0.0, rss0, aic0, bic0))

In [284]:
subset_selection()

Model      Beta0   Beta1   Beta2   Beta3     RSS      AIC      BIC
x1/x2/x3   0.979   2.007   3.035   0.011   0.452    1.160    1.313
x1/x2      0.986   2.006   3.036   0.000   0.453    1.121    1.236
x1/x3      2.184   1.756   0.000   0.296  38.415   85.022   85.136
x2/x3      2.279   0.000   2.889  -0.265  15.040   33.360   33.475
x1         2.376   1.721   0.000   0.000  38.674   85.553   85.630
x2         2.133   0.000   2.869   0.000  15.248   33.780   33.856
x3         3.278   0.000   0.000   0.041  49.650  109.810  109.887
Const      3.302   0.000   0.000   0.000  49.655  109.782  109.820
